# Quick Interface Instructions:
* Make a copy by clicking "File->Make a copy" in jupyter menu
* Give the new notebook a good new name
* Before start, make sure the circle indicating kernel status on the top-right corner is open (meaning jupyter kernel is ready)
* Run the cells below sequentially
* ** Shift-enter ** to run a cell
* When cell number is '*', the kernel is busy calculating for you. Please wait.
* Click a cell to edit

# Import useful tools

In [ ]:
# Be patient, this may take a little while
# If you see "In [*]" to the left of this cell, it means it is busy computing, just wait a bit longer
import os, imars3d, numpy as np, glob, time, pickle as pkl
from imars3d.jnbui import ct_wizard, imageslider
from imars3d.ImageFile import ImageFile

# Be patient, this may take a little while too
# %matplotlib notebook
%matplotlib inline
from matplotlib import pyplot as plt

# Configuration Wizard
The following inputs work:
* IPTS number:    15518
* Scan name:      derek_inj
* CT signature:   Derek_injec
* DF/OB files:    all files under df/ob dir

## If start from scratch, use the following cell

In [ ]:
config = ct_wizard.config()
ct_wizard.wizard(config)

## Or load previous reconstruction config. Uncomment to enable

In [ ]:
config = pkl.load(open('/HFIR/CG1D/IPTS-15518/shared/processed_data/jiaotest2/recon-config.pkl'))

## Switch to output directory

In [ ]:
%mkdir -p {config.outdir}
%cd {config.outdir}

## Save configuration

In [ ]:
pkl.dump(config, open('recon-config.pkl', 'wb'))

## Print configuration

In [ ]:
for k, v in config.__dict__.items():
    if k.startswith('_'): continue
    sv = str(v)
    if len(sv) > 60:
        sv = sv[:50] + '...'
    print "{0:20}{1:<}".format(k,sv)

# Create CT data object

In [ ]:
# import the CT class
from imars3d.CT import CT

In [ ]:
# ask for help
# CT?

In [ ]:
ct = CT(config.datadir, 
        CT_subdir=config.ct_dir, CT_identifier=config.ct_sig,
        workdir=config.workdir, outdir=config.outdir,
        ob_files=config.ob_files, df_files=config.df_files,
        # ob_identifier=ob_sig, df_identifier=df_sig
       )

## Clean up old files if desired. !!! Be very careful. Old results will be gone !!!

In [ ]:
!rm -rf {config.workdir}/*
!rm -f {config.outdir}/recon*.tiff

In [ ]:
ls {config.outdir}

# Preprocess - this takes a while

In [ ]:
%%time
ppd = ct.preprocess()

# Find region of interests (ROI)

In [ ]:
# Inspect the pre-processed image files 
# print ppd

In [ ]:
w = imageslider.ImageSlider('ppd', ppd, 400, 400)
w.show()

Please look at the image and find the ROI.
Write down the bounding box for the ROI.

** For this sample, we can define the ROI as xmin=500, ymin=0, xmax=1600, ymax=2047 **

## Run reconstruction

In [ ]:
%%time
# Now run the reconstruction. 
# Make sure to modify the crop_window and use the numbers obtained from above
# The reconstruction will take 5-60 minutes.
ct.recon(crop_window=(500,0,1600,2047))

### The following cells show several different options for reconstruction

In [ ]:
%%time
# Remove ring artifacts
ct.recon(
    crop_window=(500,0,1600,2047), 
    remove_rings_at_sinograms=True,
    )

In [ ]:
%%time
# Remove ring artifacts and smooth final reconstruction
ct.recon(
    crop_window=(500,0,1600,2047), 
    remove_rings_at_sinograms=True,
    smooth_recon=True,
    )

In [ ]:
# to see more options for reconstruction, run this cell
ct.recon?

# Check output

## The reconstruction output directory

In [ ]:
print config.outdir

## The reconstructed data

In [ ]:
w = imageslider.ImageSlider('recon', ct.r.reconstructed, 400, 400)
w.show()

## The temporay work directory

In [ ]:
print config.workdir

In [ ]:
!ls -l {config.workdir}

** Please note that the temporary working directory will be pruned after a certain period of time. If you want to retain those files, please copy them to your home directory or the experiment IPTS folder **

## Example of temp output - sinograms

In [ ]:
w2 = imageslider.ImageSlider('sinogram', ct.r.sinograms, 400, 400)
w2.show()

In [ ]:
# if sinograms was corrected, they can be viewed here
w3 = imageslider.ImageSlider('sinogram', ct.r.rar_sino, 400, 400)
w3.show()

In [ ]:
# Other temporary outputs can be viewed similarly. They are all stored in variable "ct.r". To see a list of outputs, do
print '\n'.join(name for name in dir(ct.r) if not name.startswith('_'))

# Visualization outside this notebook

ImageJ. To start, open a terminal and type
```
$ /HFIR/CG1DImaging/shared/Fiji.app/ImageJ-linux64
```